<a href="https://colab.research.google.com/github/jagadeesh-usd/receipt-expense-tracker-cv/blob/jaga-dev/notebooks/07_Evaluation_OCR_Metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Automated Expense Extraction - Receipt Parsing Using YOLO and OCR
### Evaluation Metrics (OCR & Tesseract)

#### Objective
Compare the extraction accuracy of our Rule-Based System when applied to different OCR engines.
**Configurable:** Set `EVAL_EASYOCR` or `EVAL_TESSERACT` to True/False to control which results to benchmark.

#### Metrics
* **Vendor Accuracy:** Fuzzy string matching (Similarity > 80%).
* **Date Accuracy:** Semantic date parsing (e.g., 12/01/2023 == Jan 12, 2023).
* **Total Accuracy:** Numeric comparison (within $0.10 tolerance).


#### Setup & Configuration

In [22]:
import json
import os
import pandas as pd
import numpy as np
from pathlib import Path
from difflib import SequenceMatcher
from datetime import datetime
import re
from tqdm.notebook import tqdm

In [23]:
# Check if running in Google Colab
if 'COLAB_GPU' in os.environ:
    # Mount Google Drive (for Colab)
    from google.colab import drive
    drive.mount('/content/drive')

    # Set DATA_PATH for Google Drive
    DATA_PATH = Path('/content/drive/MyDrive/data')
else:
    # Set DATA_PATH for local environment
    DATA_PATH = Path('../data')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
# CONFIGURATION FLAGS
EVAL_EASYOCR = True     # Set to True to evaluate EasyOCR results
EVAL_TESSERACT = True   # Set to True to evaluate Tesseract results

#### Ground Truth Data Handling and Matching Functions

In [25]:
GT_DIR = DATA_PATH / "raw/SROIE2019/test/entities"


def load_ground_truth(gt_folder):
    """Loads SROIE ground truth files into a Dictionary {filename: data}"""
    gt_data = {}
    files = list(Path(gt_folder).glob("*.txt"))

    print(f"   📂 Loading {len(files)} Ground Truth files...")
    for f in files:
        try:
            with open(f, 'r', encoding='utf-8') as file:
                # SROIE GT is usually a one-line JSON string inside a TXT file
                content = file.read().replace(",\n}", "}")
                data = json.loads(content)
                gt_data[f.stem] = data
        except Exception as e:
            # Fallback for non-JSON GT files (rare but possible)
            pass

    return gt_data

def is_vendor_match(pred, gt):
    if pd.isna(pred) or pd.isna(gt): return False
    # Fuzzy match: 0.8 threshold allows for small OCR typos like "Tesco" vs "Tesc0"
    return SequenceMatcher(None, str(pred).lower(), str(gt).lower()).ratio() >= 0.8

def is_date_match(pred, gt):
    if pd.isna(pred) or pd.isna(gt): return False
    try:
        # Normalize separators
        def clean_date(d): return str(d).replace('-', '/').replace('.', '/')

        # Simple string compare first
        if clean_date(pred) == clean_date(gt): return True

        # Try parsing
        d_pred = pd.to_datetime(pred, dayfirst=True, errors='coerce')
        d_gt = pd.to_datetime(gt, dayfirst=True, errors='coerce')

        if pd.notna(d_pred) and pd.notna(d_gt):
            return d_pred == d_gt

        return False
    except:
        return False

def is_total_match(pred, gt):
    if pd.isna(pred) or pd.isna(gt): return False
    try:
        # Clean currency symbols
        def clean_num(n):
            return float(re.sub(r'[^0-9\.]', '', str(n)))

        return abs(clean_num(pred) - clean_num(gt)) < 0.10 # Tolerance of 10 cents
    except:
        return False

#### Evaluate Engine Function

In [26]:
def evaluate_engine(engine_name, input_folder_name):
    print(f"\n{'='*40}")
    print(f"   EVALUATING ENGINE: {engine_name.upper()}")
    print(f"{'='*40}")

    # 1. Load Predictions
    pred_file = DATA_PATH / f"processed/extracted/{input_folder_name}/test_extracted.csv"
    if not pred_file.exists():
        print(f"Prediction file not found: {pred_file}")
        print("   Did you run Notebook 03 for this engine?")
        return

    df_pred = pd.read_csv(pred_file)
    print(f"   📄 Loaded predictions for {len(df_pred)} receipts.")

    # 2. Load Ground Truth
    gt_data = load_ground_truth(GT_DIR)

    # 3. Compare
    results = []

    for _, row in df_pred.iterrows():
        file_id = str(row['image'])

        if file_id not in gt_data:
            continue # Skip if no GT available

        gt = gt_data[file_id]

        # SROIE GT keys are usually: "company", "date", "total"
        # Our prediction keys are: "vendor", "date", "total"

        res = {
            "vendor_match": is_vendor_match(row['vendor'], gt.get('company')),
            "date_match": is_date_match(row['date'], gt.get('date')),
            "total_match": is_total_match(row['total'], gt.get('total'))
        }
        results.append(res)

    # 4. Aggregate Metrics
    if not results:
        print(" No matching records found for evaluation.")
        return

    df_res = pd.DataFrame(results)

    print(f"\n PERFORMANCE RESULTS ({engine_name}):")
    print(f"   ----------------------------------------")
    print(f"   Vendor Accuracy:  {df_res['vendor_match'].mean():.2%}")
    print(f"   Date Accuracy:    {df_res['date_match'].mean():.2%}")
    print(f"   Total Accuracy:   {df_res['total_match'].mean():.2%}")
    print(f"   ----------------------------------------")

    return df_res

#### Metrics Summary Evaluation

In [27]:
metrics_summary = {}

if EVAL_EASYOCR:
    # Expects output in 'processed/extracted/ocr'
    df_easy = evaluate_engine("EasyOCR", "ocr")
    if df_easy is not None:
        metrics_summary["EasyOCR"] = {
            "Vendor": df_easy['vendor_match'].mean(),
            "Date": df_easy['date_match'].mean(),
            "Total": df_easy['total_match'].mean()
        }

if EVAL_TESSERACT:
    # Expects output in 'processed/extracted/tesseract_ocr'
    df_tess = evaluate_engine("Tesseract", "tesseract_ocr")
    if df_tess is not None:
        metrics_summary["Tesseract"] = {
            "Vendor": df_tess['vendor_match'].mean(),
            "Date": df_tess['date_match'].mean(),
            "Total": df_tess['total_match'].mean()
        }




   EVALUATING ENGINE: EASYOCR
   📄 Loaded predictions for 347 receipts.
   📂 Loading 347 Ground Truth files...


/tmp/ipython-input-2515639541.py:38: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  d_pred = pd.to_datetime(pred, dayfirst=True, errors='coerce')



 PERFORMANCE RESULTS (EasyOCR):
   ----------------------------------------
   Vendor Accuracy:  39.48%
   Date Accuracy:    36.60%
   Total Accuracy:   40.92%
   ----------------------------------------

   EVALUATING ENGINE: TESSERACT
   📄 Loaded predictions for 347 receipts.
   📂 Loading 347 Ground Truth files...


/tmp/ipython-input-2515639541.py:38: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  d_pred = pd.to_datetime(pred, dayfirst=True, errors='coerce')



 PERFORMANCE RESULTS (Tesseract):
   ----------------------------------------
   Vendor Accuracy:  37.46%
   Date Accuracy:    36.89%
   Total Accuracy:   47.84%
   ----------------------------------------


#### Comparison Result

In [28]:
if metrics_summary:
    print("\n\n🏆 FINAL COMPARISON: EasyOCR vs Tesseract")
    comp_df = pd.DataFrame(metrics_summary).T

    # Format as percentage
    print(comp_df)

    # Save report
    comp_df.to_csv(DATA_PATH / "processed/ocr_comparison_report.csv")
    print("\n Report saved to processed/ocr_comparison_report.csv")



🏆 FINAL COMPARISON: EasyOCR vs Tesseract
             Vendor      Date     Total
EasyOCR    0.394813  0.365994  0.409222
Tesseract  0.374640  0.368876  0.478386

 Report saved to processed/ocr_comparison_report.csv
